In [1]:
%%time
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add integration folder to current working path
sys.path.append(parent_directory+"/Src/integration")

# add utils folder to current working path
sys.path.append(parent_directory+"/Src/utils")

# add FTC folder to current working path
sys.path.append(parent_directory+"/Demos/FastTensorlineComputation")

Wall time: 0 ns


# Import Agulhas Data

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../Data/AVISO/Agulhas_AVISO.mat')

U = mat_file['u'] #u
V = mat_file['v'] #v
x = mat_file['x'] #x
y = mat_file['y'] #y
time_data = mat_file['t']

Wall time: 107 ms


# Data/Parameters for Dynamical System

In [3]:
import numpy as np

# number of cores to be used for parallel computing
Ncores = 8

# time resolution of data
dt_data = time_data[0, 1]-time_data[0,0]

# periodic boundary conditions
periodic_x = False
periodic_y = False
periodic = [periodic_x, periodic_y]

# unsteady velocity field
bool_unsteady = True

# defined domain
defined_domain = np.isfinite(U[:,:,0]).astype(int)

## compute meshgrid of dataset
X, Y = np.meshgrid(x, y)

## resolution of meshgrid
dx_data = X[0,1]-X[0,0]
dy_data = Y[1,0]-Y[0,0]

delta = [dx_data, dy_data]

# Spatio-Temporal Domain of Dynamical System

In [4]:
%%time
# Time
t_OECS = 0

# store time in array
time = np.array([t_OECS])

# domain boundary (in degrees)
xmin = -3
xmax = 1
ymin = -32
ymax = -24

# spacing of meshgrid (in degrees)
dx = 0.01
dy = 0.01

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

Wall time: 998 µs


# Interpolate Velocity

In order to evaluate the velocity field at arbitrary locations and times, we must interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". In order to favour a smooth velocity field, we interpolate the velocity field in space using a cubic interpolant.

In [5]:
%%time
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, time_data, method = "cubic")

Wall time: 239 ms


# Rate of strain tensor over meshgrid of initial conditions

The rate of strain tensor $ S(\mathbf{x}, t) $ at time $ t $ is computed by iterating over meshgrid. The method *DS._spin_tensor(x, t)* computes the rate of strain tensor at point $ \mathbf{x} $  at time $ t $ by using an auxiliary meshgrid. 'aux_grid' specifies the ratio between the auxiliary grid and the original meshgrid. This parameter is generally chosen to be between $ [\dfrac{1}{10}, \dfrac{1}{100}] $. The computations are parallelized.

In [6]:
%%time
# Import package for progress bar
from tqdm.notebook import tqdm

# Import package for parallel computing
from joblib import Parallel, delayed

# Import gradient of velocity function
from ipynb.fs.defs.gradient_velocity import gradient_velocity

# Import Rate of Strain function
from ipynb.fs.defs.RateStrain import RateStrain

# Import eigenvalue/eigenvector calculator
from ipynb.fs.defs.eigen import eigen

# Define ratio of auxiliary grid spacing vs original grid_spacing
aux_grid_ratio = .1 # [1/10, 1/100]
aux_grid = [aux_grid_ratio*(X_domain[0, 1]-X_domain[0, 0]), aux_grid_ratio*(Y_domain[1, 0]-Y_domain[0, 0])]

def parallel_S(i):
    
    S_parallel = np.zeros((X_domain.shape[1], 2, 2))
    
    for j in range(S_parallel.shape[0]):
        
        x = np.array([X_domain[i,j], Y_domain[i,j]])
        
        # Compute gradient of velocity
        grad_vel = gradient_velocity(t_OECS, x, X, Y, Interpolant, periodic, defined_domain, bool_unsteady, dt_data, delta, aux_grid)
        
        # Compute rate of strain at 'x'
        S_parallel[j, :, :] = RateStrain(grad_vel)
    
    return S_parallel

S = np.array(Parallel(n_jobs=Ncores, verbose = 0)(delayed(parallel_S)(i) for i in tqdm(range(X_domain.shape[0]))))

  0%|          | 0/802 [00:00<?, ?it/s]

Wall time: 43.1 s


## Compute Tensorfield properties

We now compute the properties of the rate of strain tensor 'S' such as the eigenvalues 's1', 's2' and eigenvectors 'eigenv1', 'eigenv2'. Furthermore, we also need the spatial derivatives of the elements of 'S'.

In [7]:
# Import (cubic) RectBivariateSpline from scipy
from scipy.interpolate import RectBivariateSpline as RBS
        
# Compute gradients of elements of rate of strain tensor

S11 = np.nan_to_num(S[:,:,0,0], nan=0.0)
S12 = np.nan_to_num(S[:,:,0,1], nan=0.0)
S22 = np.nan_to_num(S[:,:,1,1], nan=0.0)

# eigenvalue of S
s = np.zeros((S.shape[0], S.shape[1]))*np.nan

# Interpolate elements of rate of strain tensor (per default: cubic)
interp_S11 = RBS(Y_domain[:,0], X_domain[0,:], S11)
interp_S12 = RBS(Y_domain[:,0], X_domain[0,:], S12)
interp_S22 = RBS(Y_domain[:,0], X_domain[0,:], S22)

# Initialize arrays
S11x = np.zeros((S.shape[0], S.shape[1]))*np.nan
S11y = np.zeros((S.shape[0], S.shape[1]))*np.nan
S12x = np.zeros((S.shape[0], S.shape[1]))*np.nan
S12y = np.zeros((S.shape[0], S.shape[1]))*np.nan
S22x = np.zeros((S.shape[0], S.shape[1]))*np.nan
S22y = np.zeros((S.shape[0], S.shape[1]))*np.nan
    
# Define auxilary meshgrid
rho_x = aux_grid[0]
rho_y = aux_grid[1]
    
#iterate over meshgrid
for i in range(X_domain.shape[0]):
        
    for j in range(Y_domain.shape[1]):
            
        x = [X_domain[i,j], Y_domain[i, j]]
        
        # compute derivatives using auxiliary grid and finite-differencing
        S11x[i, j] = (interp_S11(x[1], x[0]+rho_x)[0][0]-interp_S11(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
        S11y[i, j] = (interp_S11(x[1]+rho_y, x[0])[0][0]-interp_S11(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
        
        S12x[i, j] = (interp_S12(x[1], x[0]+rho_x)[0][0]-interp_S12(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
        S12y[i, j] = (interp_S12(x[1]+rho_y, x[0])[0][0]-interp_S12(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
            
        S22x[i, j] = (interp_S22(x[1], x[0]+rho_x)[0][0]-interp_S22(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
        S22y[i, j] = (interp_S22(x[1]+rho_y, x[0])[0][0]-interp_S22(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
        
        # compute maximum eigenvalue
        s = eig

## Interpolate $ \dot{\phi}(x, y, \phi), s(x, y) $ 

In [11]:
%%time
# Import phi_prime
from ipynb.fs.defs.phi_prime_OECS import _phi_prime_OECS

# Interpolant phi_phrime
interp_phi_prime_attracting = _phi_prime_OECS(X_domain, Y_domain, -s, S11x, S11y, S12x, S12y, S22x, S22y, Ncores)

# Interpolant phi_phrime
interp_phi_prime_repelling = _phi_prime_OECS(X_domain, Y_domain, s, S11x, S11y, S12x, S12y, S22x, S22y, Ncores)

# Interpolant s
from scipy.interpolate import RectBivariateSpline as RBS

interp_s = RBS(Y_domain[:,0], X_domain[0,:], np.nan_to_num(s, nan = 0), kx=1, ky=1)

NameError: name 's2' is not defined

# Find objective Saddle-points

In [10]:
from ipynb.fs.defs.loc_max import _loc_max

# minimum distance between objective saddles (=local maxima in the s2-field)
min_distance = .1

# indices, positions and value (of s2-field) of the objective saddles
loc_idx_x, loc_idx_y, loc_max_x, loc_max_y, loc_max_field = _loc_max(min_distance, X_domain, Y_domain, s)

NameError: name 's' is not defined

In [ ]:
###################################### PLOT OBJECTIVE SADDLES ######################################
import matplotlib.pyplot as plt

fig = plt.figure(figsize = (16, 10))
ax = plt.axes()

cax = ax.contourf(X_domain, Y_domain, s2, levels = 400, cmap = "Blues")

ax.scatter(loc_max_x, loc_max_y, c = 'y', marker = "X", s = 100)
    
ax.set_xlabel('long', fontsize = 16)
ax.set_ylabel('lat', fontsize = 16)

ax.set_xticks(np.arange(np.min(X_domain), np.max(X_domain), .5))
ax.set_yticks(np.arange(np.min(Y_domain), np.max(Y_domain), .5))

ax.set_title('Objective Saddles', fontsize = 20)

cbar = plt.colorbar(cax)
cbar.set_ticks(np.arange(0, 1.5, 0.25))
cbar.set_label(r'$ |s_1| $', rotation = 0, labelpad = 20, fontsize = 16)

plt.show()

# Compute Initial Conditions ($ x_0, y_0, \phi_0 $) for Hyperbolic OECS

In [ ]:
from math import atan, cos, sin

# initial positions from where to start integration of hyperbolic OECS
x0s1 = loc_max_x
y0s1 = loc_max_y

# Choose vector-field
v = v2.copy()
phi0s1_repelling, phi0s1_attracting = [], []

for i in range(len(loc_idx_x)):
    eigenvector = v[loc_idx_y[i], loc_idx_x[i], :]
    angle = atan(eigenvector[1]/eigenvector[0])%(2*np.pi)
    phi0s1_repelling.append(angle - np.pi/2)
    phi0s1_attracting.append(angle)

# Compute Attracting/Repelling OECS

In [ ]:
from scipy.integrate import solve_ivp
from ipynb.fs.defs.tensorline_equation import _tensorline_equation
from ipynb.fs.defs.extract_hyperbolic_OECS import _extract_hyperbolic_OECS

# define domain where velocity field is defined
defined_domain = np.isfinite(s2).astype(int)

# If tensorline is close to singularity 
# --> integration stops as eigenvector are no longer well defined.
# set threshold on rate of attraction which controls stopping criterion of integration.
s_threshold = 0.001

# define lists containing the position and the rate of attracion/repulsion along 
# unstable/stable segments of the hyperbolic OECS
x_attracting, y_attracting, s_attracting = [], [], []
x_repelling, y_repelling, s_repelling = [], [], []

# objective saddle location
objective_saddle = []

# Iterate over objective saddle-points (hyperbolic OECS)
for i in tqdm(range(len(x0s1))):
    
    # backward (-1) and forward (+1) integration
    for sign in [-1, 1]:
        
        # Append objective saddle
        objective_saddle.append([x0s1[i], y0s1[i]])
        
        # define integration interval of dummy variable 's'
        t = [0, sign*5]
        
        # define resolution of stable/unstable segments of hyperbolic OECS
        t_eval = np.linspace(t[0], t[1], 1000)

        # IC repelling OECS
        x0_repelling = [x0s1[i], y0s1[i], phi0s1_repelling[i]]
        
        # IC repelling OECS
        x0_attracting = [x0s1[i], y0s1[i], phi0s1_attracting[i]]
        
        # solve ODE for repelling OECS
        solREP = solve_ivp(_tensorline_equation, t, x0_repelling, 'BDF', t_eval, rtol=1e-8, atol=1e-8, args=(interp_phi_prime_repelling, s_threshold, interp_s1, X_domain, Y_domain, defined_domain))         
        
        # solve ODE for attracting OECS
        solATTR = solve_ivp(_tensorline_equation, t, x0_attracting, 'BDF', t_eval, rtol=1e-8, atol=1e-8, args=(interp_phi_prime_attracting, s_threshold, interp_s2, X_domain, Y_domain, defined_domain))   
        
        # extract attracting/repelling segments from the solution of the ODE as those segments
        # where the rate of attraction/repulsion |s1|=|s2| is monotonically decreasing
        x_repelling_OECS, y_repelling_OECS, s_repelling_OECS = _extract_hyperbolic_OECS(solREP.y[0,:], solREP.y[1,:], interp_s1) 
        x_attracting_OECS, y_attracting_OECS, s_attracting_OECS = _extract_hyperbolic_OECS(solATTR.y[0,:], solATTR.y[1,:], interp_s1) 
        
        x_attracting.append(x_attracting_OECS)
        y_attracting.append(y_attracting_OECS)
        s_attracting.append(s_attracting_OECS)
        
        x_repelling.append(x_repelling_OECS)
        y_repelling.append(y_repelling_OECS)
        s_repelling.append(s_repelling_OECS)

In [ ]:
################################ PLOT HYPERBOLIC OECS ################################

import matplotlib.pyplot as plt

# generate figure and axis object
fig = plt.figure(figsize = (16, 12))
ax = plt.axes()

# Plot s2-field
cax = ax.contourf(X_domain, Y_domain, s2, levels = 600, cmap = "gist_rainbow_r")

cbar = plt.colorbar(cax)
cbar.set_ticks(np.arange(0, 1.5, 0.25))
cbar.set_label(r'$ |s_1| $', rotation = 0, labelpad = 20, fontsize = 16)

# Plot attracting OECS
for i in range(len(x_attracting)):
    ax_attracting = ax.plot(x_attracting[i], y_attracting[i], c = "k", linewidth = 2.5)

# Plot repelling OECS
for j in range(len(x_repelling)):
    ax_repelling = ax.plot(x_repelling[j], y_repelling[j], c = "w", linewidth = 2.5)

# set limits
ax.set_xlim(np.min(X_domain), np.max(X_domain))
ax.set_ylim(np.min(Y_domain), np.max(Y_domain))

# plot e2 vector-field
ax.quiver(X_domain[::4,::4], Y_domain[::4,::4], v2[::4,::4,0], v2[::4,::4,1])

# Set title
ax.set_title('Repelling/Attracting OECSs from Fast Computation of Tensorlines (FCOT)', fontsize = 20)

# Plot legend
import matplotlib.patches as mpatches
attracting = mpatches.Patch(color='k', label='attracting OECS')
repelling = mpatches.Patch(color='w', label='repelling OECS')
plt.legend(handles=[attracting, repelling], fontsize = 16)
 
# Show plot
plt.show()

# Validation of Tensorline-calculation

In [ ]:
## This function evaluates the oriented vector-field at point x

def _orient_vector_field(X, Y, x, vector_field):
    
        # import Rectangular bivariate spline from scipy
        from scipy.interpolate import RectBivariateSpline as RBS
            
        # Check for orientational discontinuity by introducing appropriate scaling
        idx_x = np.searchsorted(X[0,:], x[0])
        idx_y = np.searchsorted(Y[:,0], x[1])
        
        if 0 < idx_x < X.shape[1] and 0 < idx_y < Y.shape[0]:
    
            X_reduced, Y_reduced = X[idx_y-1:idx_y+1, idx_x-1:idx_x+1], Y[idx_y-1:idx_y+1, idx_x-1:idx_x+1]
    
            vx_grid = np.array([[vector_field[idx_y-1,idx_x-1, 0], vector_field[idx_y, idx_x-1, 0]],
                      [vector_field[idx_y-1,idx_x, 0], vector_field[idx_y, idx_x, 0]]])
            vy_grid = np.array([[vector_field[idx_y-1,idx_x-1, 1], vector_field[idx_y, idx_x-1, 1]],
                      [vector_field[idx_y-1,idx_x, 1], vector_field[idx_y, idx_x, 1]]])
        
            for i in range(2):
                for j in range(2):
                    if vx_grid[0, 0]*vx_grid[i, j]+vy_grid[0, 0]*vy_grid[i, j] < 0:
                        vx_grid[i, j] = -vx_grid[i, j]
                        vy_grid[i, j] = -vy_grid[i, j]
    
            vx_Interp = RBS(Y_reduced[:,0], X_reduced[0,:], vx_grid, kx = 1, ky = 1)
            vy_Interp = RBS(Y_reduced[:,0], X_reduced[0,:], vy_grid, kx = 1, ky = 1)
        
            return vx_Interp(x[1], x[0])[0][0], vy_Interp(x[1], x[0])[0][0]
        
        else:
            
            return None, None

## Validation of attracting OECS

In [ ]:
distance = []
diff_phi = []

for i in range(len(x_attracting)):
    for j in range(1, len(x_attracting[i])-1):
        
        dy = y_attracting[i][j+1]-y_attracting[i][j-1]
        dx = x_attracting[i][j+1]-x_attracting[i][j-1]
        
        phi = atan(dy/dx)
        
        x = [x_attracting[i][j], y_attracting[i][j]]
        
        # evaluate oriented vector field v2 at x
        vector = _orient_vector_field(X_domain, Y_domain, x, v2)
        
        if vector[0] is not None:
            dy_oriented = vector[1]
            dx_oriented = vector[0]
        
            phi_True = atan(dy_oriented/dx_oriented)
            
            diff_phi.append(min(np.abs(-phi-phi_True), np.abs(phi-phi_True))/(np.pi/2))
            
            distance.append(sqrt((x[0]-objective_saddle[i][0])**2+(x[1]-objective_saddle[i][1])**2))
           

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.axes()

ax.set_title(r'Error of $ \phi $ vs distance from core of saddle for attracting OECS', fontsize = 16)

ax.scatter(distance, diff_phi, marker = "^", s = 10)

ax.set_ylabel(r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} $', fontsize = 16)
ax.set_xlabel("distance from core of saddle (deg)", fontsize = 16)

ax.text(0, 0.7, r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} < .1$: '+str(np.around(len([i for i in diff_phi if i < .1])/len(diff_phi),2)*100)+r'%', fontsize = 16)
ax.text(0, 0.6, r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} < .05$: '+str(np.around(len([i for i in diff_phi if i < .05])/len(diff_phi),2)*100)+r'%', fontsize = 16)
ax.text(0, 0.5, r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} < .01$: '+str(np.around(len([i for i in diff_phi if i < .01])/len(diff_phi),2)*100)+r'%', fontsize = 16)
plt.show()

## Validation of repelling OECS

In [ ]:
distance = []
diff_phi = []

for i in range(len(x_repelling)):
    for j in range(1, len(x_repelling[i])-1):
        
        dy = y_repelling[i][j+1]-y_repelling[i][j-1]
        dx = x_repelling[i][j+1]-x_repelling[i][j-1]
        
        phi = atan(dy/dx)
        
        x = [x_repelling[i][j], y_repelling[i][j]]
        
        # evaluate oriented vector field v2 at x
        vector = _orient_vector_field(X_domain, Y_domain, x, v1)
        
        if vector[0] is not None:
            dy_oriented = vector[1]
            dx_oriented = vector[0]
        
            phi_True = atan(dy_oriented/dx_oriented)
            
            diff_phi.append(min(np.abs(-phi-phi_True), np.abs(phi-phi_True))/(np.pi/2))
            
            distance.append(sqrt((x[0]-objective_saddle[i][0])**2+(x[1]-objective_saddle[i][1])**2))
           

In [ ]:
fig = plt.figure(figsize=(16, 10))
ax = plt.axes()

ax.set_title(r'Error of $ \phi $ vs distance from core of saddle for repelling OECS', fontsize = 16)

ax.scatter(distance, diff_phi, marker = "^", s = 10)

ax.set_ylabel(r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} $', fontsize = 16)
ax.set_xlabel("distance from core of saddle (deg)", fontsize = 16)

ax.text(0, 0.7, r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} < .1$: '+str(np.around(len([i for i in diff_phi if i < .1])/len(diff_phi),2)*100)+r'%', fontsize = 16)
ax.text(0, 0.6, r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} < .05$: '+str(np.around(len([i for i in diff_phi if i < .05])/len(diff_phi),2)*100)+r'%', fontsize = 16)
ax.text(0, 0.5, r'$ \dfrac{|\phi-\phi_{oriented}|}{\pi/2} < .01$: '+str(np.around(len([i for i in diff_phi if i < .01])/len(diff_phi),2)*100)+r'%', fontsize = 16)
plt.show()